In [1]:
import argparse
import gym
import numpy as np
from itertools import count

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.distributions import Categorical

In [2]:
env = gym.make('CartPole-v0')
gamma = 0.99
# env.seed(args.seed)
# torch.manual_seed(args.seed)

In [3]:
class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.affine1 = nn.Linear(4, 128)
        self.affine2 = nn.Linear(128, 2)

        self.saved_log_probs = []
        self.rewards = []

    def forward(self, x):
        x = F.relu(self.affine1(x))
        action_scores = self.affine2(x)
        return F.softmax(action_scores, dim=1)

In [4]:
def select_action(state):
    state = torch.from_numpy(state).float().unsqueeze(0)
    probs = policy(state) #returns the probability of an action with the given state (eg. output of the NN)
    m = Categorical(probs)
    action = m.sample()
    policy.saved_log_probs.append(m.log_prob(action))
    return action.item()

In [5]:
def finish_episode():
    R = 0
    policy_loss = []
    rewards = []
    for r in policy.rewards[::-1]:
        R = r + gamma * R
        rewards.insert(0, R)
    rewards = torch.tensor(rewards)
    rewards = (rewards - rewards.mean()) / (rewards.std() + eps)
    for log_prob, reward in zip(policy.saved_log_probs, rewards):
        policy_loss.append(-log_prob * reward)
    optimizer.zero_grad()
    policy_loss = torch.cat(policy_loss).sum()
    policy_loss.backward()
    optimizer.step()
    del policy.rewards[:]
    del policy.saved_log_probs[:]

In [6]:
running_reward = 10 # what is the running reward?
policy = Policy()
# optimizer updates the weights of the NN
optimizer = optim.Adam(policy.parameters(), lr=1e-2)
eps = np.finfo(np.float32).eps.item()

In [21]:
state = env.reset()

In [23]:
action = select_action(state)
action

0

In [24]:
state, reward, done, _ = env.step(action)

In [25]:
state

array([-0.04034541, -0.1804821 ,  0.0074924 ,  0.29441308])

In [26]:
reward

1.0

In [27]:
done

False

In [28]:
policy.rewards.append(reward)

In [29]:
policy.rewards

[1.0]

In [44]:
for i_episode in range(100): #count(1):
    state = env.reset()
    for t in range(10000):  # 10000 steps per episode, when the episode is done, done = True
        action = select_action(state) # select action
        state, reward, done, _ = env.step(action)

        policy.rewards.append(reward)
        if done:
             break

#     running_reward = running_reward * 0.99 + t * 0.01
#     finish_episode()
#     print("finish episode")

In [45]:
R = 0
policy_loss = []
rewards = []


In [49]:
len(policy.saved_log_probs)

1359756

In [46]:
for r in policy.rewards[::-1]:
    R = r + gamma * R
    rewards.insert(0, R)


KeyboardInterrupt: 

In [ ]:
rewards

In [ ]:
rewards = torch.tensor(rewards)
rewards = (rewards - rewards.mean()) / (rewards.std() + eps)


In [ ]:
for log_prob, reward in zip(policy.saved_log_probs, rewards):
    policy_loss.append(-log_prob * reward)


In [ ]:
optimizer.zero_grad()

In [ ]:
policy_loss = torch.cat(policy_loss).sum()


In [ ]:
policy_loss.backward()


In [ ]:
optimizer.step()
del policy.rewards[:]
del policy.saved_log_probs[:]